Imports and engine

In [2]:
!pip install pandas sqlalchemy matplotlib

  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/69/ac/b42ad16800d0885105b59380ad69aad0cce5a65276e269ce2729a2343b6a/sqlalchemy-2.0.46-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd

# If the notebook lives in etl-ai-schema/notebooks, CWD is that folder:
# project_root = parent of the notebooks directory.
project_root = Path.cwd().parent   # should be .../etl-ai-schema
db_path = project_root / "data" / "database.db"

print("Project root:", project_root)
print("DB path:", db_path)

engine = create_engine(f"sqlite:///{db_path}")
engine

Project root: /Users/rajakarthikchirumamilla/Documents/ThesisWork/etl-ai-schema
DB path: /Users/rajakarthikchirumamilla/Documents/ThesisWork/etl-ai-schema/data/database.db


Engine(sqlite:////Users/rajakarthikchirumamilla/Documents/ThesisWork/etl-ai-schema/data/database.db)

List tables

In [13]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", engine)
tables

,name
0,yellow_trips_v1
1,yellow_trips_v2


Load V1 and V2

In [14]:
df_v1 = pd.read_sql("SELECT * FROM yellow_trips_v1", engine)
df_v2 = pd.read_sql("SELECT * FROM yellow_trips_v2", engine)

len(df_v1), len(df_v2), df_v1.columns, df_v2.columns

(99376,
 99376,
 Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
        'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
        'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
        'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
        'total_amount', 'congestion_surcharge', 'airport_fee',
        'trip_duration_minutes', 'trip_revenue'],
       dtype='str'),
 Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
        'passenger_count', 'trip_km', 'RatecodeID', 'store_and_fwd_flag',
        'PULocationID', 'DOLocationID', 'fare_amount', 'extra', 'mta_tax',
        'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
        'congestion_surcharge', 'airport_fee', 'tip_ratio',
        'trip_duration_minutes', 'trip_revenue'],
       dtype='str'))

Focus on key metrics

In [15]:
print("V1 columns:", df_v1.columns.tolist())
print("V2 columns:", df_v2.columns.tolist())

V1 columns: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee', 'trip_duration_minutes', 'trip_revenue']
V2 columns: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_km', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee', 'tip_ratio', 'trip_duration_minutes', 'trip_revenue']


In [16]:
# Align distance column names for comparison
df_v1_common = df_v1.copy()
df_v2_common = df_v2.copy()

# In V2, copy trip_km into a column named trip_distance so we can use same metric name
df_v2_common["trip_distance"] = df_v2_common["trip_km"]

metrics = ["fare_amount", "trip_distance"]

df_v1_metrics = df_v1_common[metrics].copy()
df_v2_metrics = df_v2_common[metrics].copy()

print("=== V1 (baseline) ===")
display(df_v1_metrics.describe())

print("=== V2 (after evolution, AI-assisted) ===")
display(df_v2_metrics.describe())

=== V1 (baseline) ===


,fare_amount,trip_distance
count,99376.000000,99376.000000
mean,11.858665,2.798451
std,9.851564,3.407493
min,0.010000,0.010000
25%,6.500000,1.000000
50%,9.000000,1.680000
75%,13.500000,3.000000
max,285.000000,71.900000


=== V2 (after evolution, AI-assisted) ===


,fare_amount,trip_distance
count,99376.000000,99376.000000
mean,11.858665,2.798451
std,9.851564,3.407493
min,0.010000,0.010000
25%,6.500000,1.000000
50%,9.000000,1.680000
75%,13.500000,3.000000
max,285.000000,71.900000
